# SparkSQL and DataFrames 

## RDDs, DataSets, and DataFrames

RDDs are the original interface for Spark programming.

DataFrames were introduced in 1.3

Datasets were introduced in 1.6, and unified with DataFrames in 2.0

### Advantages of DataFrames:

from https://www.datacamp.com/community/tutorials/apache-spark-python:

> More specifically, the performance improvements are due to two things, which you’ll often come across when you’re reading up DataFrames: custom memory management (project Tungsten), which will make sure that your Spark jobs much faster given CPU constraints, and optimized execution plans (Catalyst optimizer), of which the logical plan of the DataFrame is a part.

## SparkSQL and DataFrames 


pyspark does not have the Dataset API, which is available only if you use Spark from a statically typed language: Scala or Java.

From https://spark.apache.org/docs/2.2.0/sql-programming-guide.html:

> A DataFrame is a Dataset organized into named columns. It is conceptually equivalent to a table in a relational database or a data frame in R/Python, but with richer optimizations under the hood. DataFrames can be constructed from a wide array of sources such as: structured data files, tables in Hive, external databases, or existing RDDs. The DataFrame API is available in Scala, Java, Python, and R. In Scala and Java, a DataFrame is represented by a Dataset of Rows. In the Scala API, DataFrame is simply a type alias of Dataset[Row]. While, in Java API, users need to use Dataset&lt;Row> to represent a DataFrame.


### The pyspark.sql module

Important classes of Spark SQL and DataFrames:

* `pyspark.sql.SparkSession` Main entry point for DataFrame and SQL functionality.

* `pyspark.sql.DataFrame` A distributed collection of data grouped into named columns.

* `pyspark.sql.Column` A column expression in a DataFrame.

* `pyspark.sql.Row` A row of data in a DataFrame.

* `pyspark.sql.GroupedData` Aggregation methods, returned by DataFrame.groupBy().

* `pyspark.sql.DataFrameNaFunctions` Methods for handling missing data (null values).

* `pyspark.sql.DataFrameStatFunctions` Methods for statistics functionality.

* `pyspark.sql.functions` List of built-in functions available for DataFrame.

* `pyspark.sql.types` List of data types available.

* `pyspark.sql.Window` For working with window functions.

http://spark.apache.org/docs/2.2.0/api/python/pyspark.sql.html

https://spark.apache.org/docs/2.2.0/sql-programming-guide.html

## SparkSession

The traditional way to interact with Spark is the SparkContext. In the notebooks we get that from the pyspark driver.

From 2.0 we can use SparkSession to replace SparkConf, SparkContext and SQLContext

In [1]:
sc

''

In [2]:
from pyspark.sql import SparkSession
session= SparkSession.builder.getOrCreate()

In [3]:
session

#### Passing other options to spark session:
    
    

In [4]:
session = SparkSession.builder.config('someoption.key','somevalue').getOrCreate()

We can check option values in the resulting session like this:

In [5]:
session.sparkContext.getConf().getAll()

[('spark.sql.catalogImplementation', 'hive'),
 ('someoption.key', 'somevalue'),
 ('spark.rdd.compress', 'True'),
 ('spark.serializer.objectStreamReset', '100'),
 ('spark.master', 'local[*]'),
 ('spark.executor.id', 'driver'),
 ('spark.submit.deployMode', 'client'),
 ('spark.driver.host', '10.0.2.15'),
 ('spark.app.id', 'local-1557568010788'),
 ('spark.app.name', 'PySparkShell'),
 ('spark.ui.showConsoleProgress', 'true'),
 ('spark.driver.port', '34093')]

### Creating DataFrames

SparkSession.createDataFrame: from an RDD, a list or a pandas.DataFrame.

In [8]:
# Vamos a crear datos para un DataFrame
import random
random.choice(['a','b','c'])

'c'

In [9]:
random.seed(42)
ids= range(12)
positions=[random.choice(['mechanic','sales','manager']) for id_ in ids]
positions

['manager',
 'mechanic',
 'mechanic',
 'manager',
 'sales',
 'mechanic',
 'mechanic',
 'mechanic',
 'manager',
 'mechanic',
 'manager',
 'manager']

In [10]:
# Unir ids con positions con zip
rows = zip(ids,positions)
df = session.createDataFrame(rows)
df

DataFrame[_1: bigint, _2: string]

In [11]:
df.show()

+---+--------+
| _1|      _2|
+---+--------+
|  0| manager|
|  1|mechanic|
|  2|mechanic|
|  3| manager|
|  4|   sales|
|  5|mechanic|
|  6|mechanic|
|  7|mechanic|
|  8| manager|
|  9|mechanic|
| 10| manager|
| 11| manager|
+---+--------+



In [18]:
df.collect()

[Row(_1=0, _2='manager'),
 Row(_1=1, _2='mechanic'),
 Row(_1=2, _2='mechanic'),
 Row(_1=3, _2='manager'),
 Row(_1=4, _2='sales'),
 Row(_1=5, _2='mechanic'),
 Row(_1=6, _2='mechanic'),
 Row(_1=7, _2='mechanic'),
 Row(_1=8, _2='manager'),
 Row(_1=9, _2='mechanic'),
 Row(_1=10, _2='manager'),
 Row(_1=11, _2='manager')]

In [12]:
df.printSchema()

root
 |-- _1: long (nullable = true)
 |-- _2: string (nullable = true)



In [20]:
from pyspark.sql import Row

In [21]:
help(Row)

Help on class Row in module pyspark.sql.types:

class Row(builtins.tuple)
 |  Row(*args, **kwargs)
 |  
 |  A row in L{DataFrame}.
 |  The fields in it can be accessed:
 |  
 |  * like attributes (``row.key``)
 |  * like dictionary values (``row[key]``)
 |  
 |  ``key in row`` will search through row keys.
 |  
 |  Row can be used to create a row object by using named arguments,
 |  the fields will be sorted by names. It is not allowed to omit
 |  a named argument to represent the value is None or missing. This should be
 |  explicitly set to None in this case.
 |  
 |  >>> row = Row(name="Alice", age=11)
 |  >>> row
 |  Row(age=11, name='Alice')
 |  >>> row['name'], row['age']
 |  ('Alice', 11)
 |  >>> row.name, row.age
 |  ('Alice', 11)
 |  >>> 'name' in row
 |  True
 |  >>> 'wrong_key' in row
 |  False
 |  
 |  Row also can be used to create another Row like class, then it
 |  could be used to create Row objects, such as
 |  
 |  >>> Person = Row("name", "age")
 |  >>> Person
 |  <R

#### Otra forma de crear DF

In [22]:
rows = [Row(id=id_, position=position_) for id_, position_ in zip(ids,positions)]

In [23]:
rows

[Row(id=0, position='manager'),
 Row(id=1, position='mechanic'),
 Row(id=2, position='mechanic'),
 Row(id=3, position='manager'),
 Row(id=4, position='sales'),
 Row(id=5, position='mechanic'),
 Row(id=6, position='mechanic'),
 Row(id=7, position='mechanic'),
 Row(id=8, position='manager'),
 Row(id=9, position='mechanic'),
 Row(id=10, position='manager'),
 Row(id=11, position='manager')]

#### Otra forma de crear DF

In [24]:
df = session.createDataFrame(zip(ids,positions),schema=['id','position'])
df.show()

+---+--------+
| id|position|
+---+--------+
|  0| manager|
|  1|mechanic|
|  2|mechanic|
|  3| manager|
|  4|   sales|
|  5|mechanic|
|  6|mechanic|
|  7|mechanic|
|  8| manager|
|  9|mechanic|
| 10| manager|
| 11| manager|
+---+--------+



In [25]:
# Para pasarlo a un Rdd
myRdd = df.rdd
myRdd.take(5)

[Row(id=0, position='manager'),
 Row(id=1, position='mechanic'),
 Row(id=2, position='mechanic'),
 Row(id=3, position='manager'),
 Row(id=4, position='sales')]

### Creating DataFrames

* From RDDs
* from Hive tables
* From Spark sources: parquet (default), json, jdbc, orc, libsvm, csv, text


#### From RDDs

In [28]:
rdd = session.sparkContext.textFile('coupon150720.csv')
rdd.take(5)

['79062005698500,1,MAA,AUH,9W,9W,56.79,USD,1,H,H,0526,150904,OK,IAF0',
 '79062005698500,2,AUH,CDG,9W,9W,84.34,USD,1,H,H,6120,150905,OK,IAF0',
 '79062005924069,1,CJB,MAA,9W,9W,60.0,USD,1,H,H,2768,150721,OK,IAA0',
 '79065668570385,1,DEL,DXB,9W,9W,160.63,USD,2,S,S,0546,150804,OK,INA0',
 '79065668737021,1,AUH,IXE,9W,9W,152.46,USD,1,V,V,0501,150803,OK,INA0']

In [30]:
split_rdd = rdd.map(lambda line: line.split(','))
split_rdd.take(3)

[['79062005698500',
  '1',
  'MAA',
  'AUH',
  '9W',
  '9W',
  '56.79',
  'USD',
  '1',
  'H',
  'H',
  '0526',
  '150904',
  'OK',
  'IAF0'],
 ['79062005698500',
  '2',
  'AUH',
  'CDG',
  '9W',
  '9W',
  '84.34',
  'USD',
  '1',
  'H',
  'H',
  '6120',
  '150905',
  'OK',
  'IAF0'],
 ['79062005924069',
  '1',
  'CJB',
  'MAA',
  '9W',
  '9W',
  '60.0',
  'USD',
  '1',
  'H',
  'H',
  '2768',
  '150721',
  'OK',
  'IAA0']]

In [31]:
newDf = session.createDataFrame(split_rdd)
newDf.show()

+--------------+---+---+---+---+---+------+---+---+---+---+----+------+---+----+
|            _1| _2| _3| _4| _5| _6|    _7| _8| _9|_10|_11| _12|   _13|_14| _15|
+--------------+---+---+---+---+---+------+---+---+---+---+----+------+---+----+
|79062005698500|  1|MAA|AUH| 9W| 9W| 56.79|USD|  1|  H|  H|0526|150904| OK|IAF0|
|79062005698500|  2|AUH|CDG| 9W| 9W| 84.34|USD|  1|  H|  H|6120|150905| OK|IAF0|
|79062005924069|  1|CJB|MAA| 9W| 9W|  60.0|USD|  1|  H|  H|2768|150721| OK|IAA0|
|79065668570385|  1|DEL|DXB| 9W| 9W|160.63|USD|  2|  S|  S|0546|150804| OK|INA0|
|79065668737021|  1|AUH|IXE| 9W| 9W|152.46|USD|  1|  V|  V|0501|150803| OK|INA0|
|79062006192650|  1|RPR|BOM| 9W| 9W|  68.5|USD|  1|  K|  K|2202|150720| OK|IAE0|
|79062006192650|  2|BOM|RPR| 9W| 9W|  68.5|USD|  1|  H|  H|0377|150721| OK|IAE0|
|79062005733853|  1|DEL|DED| 9W| 9W| 56.16|USD|  1|  V|  V|2839|150801| OK|INA0|
|79062005836987|  1|ATL|LGA| AA| AA|  28.3|USD|  1|  V|  V|3237|150903| OK|INB0|
|79062005836987|  2|LGA|EWR|

In [33]:
newDf.printSchema()

root
 |-- _1: string (nullable = true)
 |-- _2: string (nullable = true)
 |-- _3: string (nullable = true)
 |-- _4: string (nullable = true)
 |-- _5: string (nullable = true)
 |-- _6: string (nullable = true)
 |-- _7: string (nullable = true)
 |-- _8: string (nullable = true)
 |-- _9: string (nullable = true)
 |-- _10: string (nullable = true)
 |-- _11: string (nullable = true)
 |-- _12: string (nullable = true)
 |-- _13: string (nullable = true)
 |-- _14: string (nullable = true)
 |-- _15: string (nullable = true)



### Inferring and specifying schemas

In [35]:
from pyspark.sql import types

types.IntegerType()

IntegerType

#### Fully specifying a schema

We need to create a `StructType` composed of `StructField`s. each of those specifies afiled with name, type and `nullable` properties. 

In [40]:
fields = [types.StructField('id',types.IntegerType()),
          types.StructField('position',types.StringType())]
my_schema = types.StructType(fields)
my_schema

StructType(List(StructField(id,IntegerType,true),StructField(position,StringType,true)))

In [42]:
session.createDataFrame(zip(ids,positions),schema=my_schema).show()

+---+--------+
| id|position|
+---+--------+
|  0| manager|
|  1|mechanic|
|  2|mechanic|
|  3| manager|
|  4|   sales|
|  5|mechanic|
|  6|mechanic|
|  7|mechanic|
|  8| manager|
|  9|mechanic|
| 10| manager|
| 11| manager|
+---+--------+



#### From csv files

We can either read them directly into dataframes or read them as RDDs and transform that into a DataFrame. This second way will be very useful if we have unstructured data like web server logs.

In [43]:
df_from_csv = session.read.csv('coupon150720.csv')
df_from_csv.show(5)

+--------------+---+---+---+---+---+------+---+---+---+----+----+------+----+----+
|           _c0|_c1|_c2|_c3|_c4|_c5|   _c6|_c7|_c8|_c9|_c10|_c11|  _c12|_c13|_c14|
+--------------+---+---+---+---+---+------+---+---+---+----+----+------+----+----+
|79062005698500|  1|MAA|AUH| 9W| 9W| 56.79|USD|  1|  H|   H|0526|150904|  OK|IAF0|
|79062005698500|  2|AUH|CDG| 9W| 9W| 84.34|USD|  1|  H|   H|6120|150905|  OK|IAF0|
|79062005924069|  1|CJB|MAA| 9W| 9W|  60.0|USD|  1|  H|   H|2768|150721|  OK|IAA0|
|79065668570385|  1|DEL|DXB| 9W| 9W|160.63|USD|  2|  S|   S|0546|150804|  OK|INA0|
|79065668737021|  1|AUH|IXE| 9W| 9W|152.46|USD|  1|  V|   V|0501|150803|  OK|INA0|
+--------------+---+---+---+---+---+------+---+---+---+----+----+------+----+----+
only showing top 5 rows



In [44]:
df_with_sql = session.sql('SELECT * from csv.`coupon150720.csv` where _c3="MAD"')

In [45]:
df_with_sql.show(5)

+--------------+---+---+---+---+---+-----+---+---+---+----+----+------+----+----+
|           _c0|_c1|_c2|_c3|_c4|_c5|  _c6|_c7|_c8|_c9|_c10|_c11|  _c12|_c13|_c14|
+--------------+---+---+---+---+---+-----+---+---+---+----+----+------+----+----+
|79062005639642|  4|BRU|MAD| UX| UX|21.02|USD|  2|  Q|   Q|1172|150912|  OK|INBQ|
|79065668754871|  6|BRU|MAD| SN| 9W|27.66|USD|  2|  W|   W|6021|151018|  OK|INBQ|
|79065668917696|  2|CDG|MAD| AF| AF|46.97|USD|  1|  T|   T|1400|150905|  OK|INB0|
|79062006133090|  2|CDG|MAD| AF| AF| 3.38|USD|  1|  X|   X|1400|151103|  OK|IAE0|
|79062006110497|  1|CDG|MAD| AF| AF|26.02|USD|  1|  Y|   Y|1300|150724|  SA|INA0|
+--------------+---+---+---+---+---+-----+---+---+---+----+----+------+----+----+
only showing top 5 rows



#### From other types of data

Apache Parquet is a free and open-source column-oriented data store of the Apache Hadoop ecosystem. It is similar to the other columnar storage file formats available in Hadoop namely RCFile and Optimized RCFile. It is compatible with most of the data processing frameworks in the Hadoop environment.

In [ ]:
# session.read.json
# session.read.parquet

### Basic operations with DataFrames

In [46]:
df.show()

+---+--------+
| id|position|
+---+--------+
|  0| manager|
|  1|mechanic|
|  2|mechanic|
|  3| manager|
|  4|   sales|
|  5|mechanic|
|  6|mechanic|
|  7|mechanic|
|  8| manager|
|  9|mechanic|
| 10| manager|
| 11| manager|
+---+--------+



In [47]:
df.take(5)

[Row(id=0, position='manager'),
 Row(id=1, position='mechanic'),
 Row(id=2, position='mechanic'),
 Row(id=3, position='manager'),
 Row(id=4, position='sales')]

In [48]:
df.printSchema()

root
 |-- id: long (nullable = true)
 |-- position: string (nullable = true)



### Filtering and selecting

Syntax inspired in SQL.

In [50]:
df.select('id').show(5)

+---+
| id|
+---+
|  0|
|  1|
|  2|
|  3|
|  4|
+---+
only showing top 5 rows



In [51]:
df.select(df['id']).show(5)

+---+
| id|
+---+
|  0|
|  1|
|  2|
|  3|
|  4|
+---+
only showing top 5 rows



In [53]:
df.select(df.id).show(5)

+---+
| id|
+---+
|  0|
|  1|
|  2|
|  3|
|  4|
+---+
only showing top 5 rows



If we want to filter, we will need to build an instance of `Column`, using square bracket notation.

In [54]:
filtered = df.filter(df['id']>5)
filtered.show()

+---+--------+
| id|position|
+---+--------+
|  6|mechanic|
|  7|mechanic|
|  8| manager|
|  9|mechanic|
| 10| manager|
| 11| manager|
+---+--------+



In [67]:
from pyspark.sql import functions as f
filtered = df.filter(f.col('id')>5)
filtered.show()

+---+--------+
| id|position|
+---+--------+
|  6|mechanic|
|  7|mechanic|
|  8| manager|
|  9|mechanic|
| 10| manager|
| 11| manager|
+---+--------+



That's because a comparison between str and int will error out, so spark will not even get the chance to infer to which column we are referring.

`where` is exactly synonimous with `filter`

In [56]:
whered = df.where(df['id']>5)
whered.show()

+---+--------+
| id|position|
+---+--------+
|  6|mechanic|
|  7|mechanic|
|  8| manager|
|  9|mechanic|
| 10| manager|
| 11| manager|
+---+--------+



A column is quite different to a Pandas Series. It is just a reference to a column, and can only be used to construct sparkSQL expressions (select, where...). It can't be collected or taken as a one-dimensional sequence:

In [59]:
col = df['id']
col.collect()

TypeError: 'Column' object is not callable

#### Exercise

Extract all employee ids which correspond to managers

In [61]:
df.select(df['id'])\
  .filter(df['position'] == 'manager').show()

+---+
| id|
+---+
|  0|
|  3|
|  8|
| 10|
| 11|
+---+



### Adding columns

Dataframes are immutable, since they are built on top of RDDs, so we can not assign to them. We need to create new DataFrames with the appropriate columns.

In [65]:
df.withColumn('newColumn',df['id']*100).show()

+---+--------+---------+
| id|position|newColumn|
+---+--------+---------+
|  0| manager|        0|
|  1|mechanic|      100|
|  2|mechanic|      200|
|  3| manager|      300|
|  4|   sales|      400|
|  5|mechanic|      500|
|  6|mechanic|      600|
|  7|mechanic|      700|
|  8| manager|      800|
|  9|mechanic|      900|
| 10| manager|     1000|
| 11| manager|     1100|
+---+--------+---------+



In [69]:
df2 = df.withColumn('newColumn',f.col('id')*100)
df2.show()

+---+--------+---------+
| id|position|newColumn|
+---+--------+---------+
|  0| manager|        0|
|  1|mechanic|      100|
|  2|mechanic|      200|
|  3| manager|      300|
|  4|   sales|      400|
|  5|mechanic|      500|
|  6|mechanic|      600|
|  7|mechanic|      700|
|  8| manager|      800|
|  9|mechanic|      900|
| 10| manager|     1000|
| 11| manager|     1100|
+---+--------+---------+



### User defined functions

There are many useful functions in pyspark.sql.functions. These work on columns, that is, they are vectorial.

We can write User Defined Functions (`udf`s), which allow us to "vectorize" operations: write a standard function to process single elements, then build a udf with that that works on columns in a DataFrame, like a SQL function.

In [70]:
from pyspark.sql import functions as f
f.sqrt(df['id'])

Column<b'SQRT(id)'>

In [71]:
df.select(f.sqrt('id')).show()

+------------------+
|          SQRT(id)|
+------------------+
|               0.0|
|               1.0|
|1.4142135623730951|
|1.7320508075688772|
|               2.0|
|  2.23606797749979|
| 2.449489742783178|
|2.6457513110645907|
|2.8284271247461903|
|               3.0|
|3.1622776601683795|
|   3.3166247903554|
+------------------+



In [72]:
df3 = df.select('id',
                'position',
                f.sqrt('id')).show()

+---+--------+------------------+
| id|position|          SQRT(id)|
+---+--------+------------------+
|  0| manager|               0.0|
|  1|mechanic|               1.0|
|  2|mechanic|1.4142135623730951|
|  3| manager|1.7320508075688772|
|  4|   sales|               2.0|
|  5|mechanic|  2.23606797749979|
|  6|mechanic| 2.449489742783178|
|  7|mechanic|2.6457513110645907|
|  8| manager|2.8284271247461903|
|  9|mechanic|               3.0|
| 10| manager|3.1622776601683795|
| 11| manager|   3.3166247903554|
+---+--------+------------------+



In [ ]:
df4 = df.select('id',
                'position',
                f.sqrt('id')).show()

In [73]:
import math
math.log1p(0)

0.0

In [74]:
# Dará error porque no es una función de pyspark que trabaja para columnas....
# Por lo tanto habrá que crear un udf
df3 = df.select('id',
                'position',
                math.log1p('id')).show()

TypeError: must be real number, not str

This errors out because 

```python
math.log1p
```

is not a udf: it doesn't know how to work with strings or Column objects:

But we can transform it into a udf:

In [75]:
udf_log1p = f.udf(math.log1p)
df.withColumn('newColUdf', udf_log1p(df['id'])).show()

+---+--------+------------------+
| id|position|         newColUdf|
+---+--------+------------------+
|  0| manager|               0.0|
|  1|mechanic|0.6931471805599453|
|  2|mechanic|1.0986122886681096|
|  3| manager|1.3862943611198906|
|  4|   sales|1.6094379124341003|
|  5|mechanic| 1.791759469228055|
|  6|mechanic|1.9459101490553132|
|  7|mechanic|2.0794415416798357|
|  8| manager|2.1972245773362196|
|  9|mechanic| 2.302585092994046|
| 10| manager|2.3978952727983707|
| 11| manager|2.4849066497880004|
+---+--------+------------------+



We can do the same with any function we dream up:

In [77]:
def odd_letters(word):
    return word[::2]

odd_letters('manager')

'mngr'

In [78]:
odd_udf = f.udf(odd_letters)

df.withColumn('newColUdf', odd_udf(df['position'])).show()

+---+--------+---------+
| id|position|newColUdf|
+---+--------+---------+
|  0| manager|     mngr|
|  1|mechanic|     mcai|
|  2|mechanic|     mcai|
|  3| manager|     mngr|
|  4|   sales|      sls|
|  5|mechanic|     mcai|
|  6|mechanic|     mcai|
|  7|mechanic|     mcai|
|  8| manager|     mngr|
|  9|mechanic|     mcai|
| 10| manager|     mngr|
| 11| manager|     mngr|
+---+--------+---------+



If we want the resulting columns to be of a particular type, we need to specify the return type. This is because in Python return types can not be inferred.

In [79]:
df3 = df.select('id',
                'position',
                 udf_log1p('id'))
df3.printSchema()

root
 |-- id: long (nullable = true)
 |-- position: string (nullable = true)
 |-- log1p(id): string (nullable = true)



In [84]:
udf_log1p_typed = f.udf(math.log1p, types.FloatType())
df.withColumn('newColUdf', udf_log1p_typed(df['id'])).show()

+---+--------+---------+
| id|position|newColUdf|
+---+--------+---------+
|  0| manager|      0.0|
|  1|mechanic|0.6931472|
|  2|mechanic|1.0986123|
|  3| manager|1.3862944|
|  4|   sales| 1.609438|
|  5|mechanic|1.7917595|
|  6|mechanic|1.9459101|
|  7|mechanic|2.0794415|
|  8| manager|2.1972246|
|  9|mechanic|2.3025851|
| 10| manager|2.3978953|
| 11| manager|2.4849067|
+---+--------+---------+



In [85]:
df.withColumn('newColUdf', udf_log1p_typed(df['id'])).printSchema()

root
 |-- id: long (nullable = true)
 |-- position: string (nullable = true)
 |-- newColUdf: float (nullable = true)



In [86]:
f.udf(lambda x: -x)

<function __main__.<lambda>(x)>

Think about this function: what is its return type?

#### Exercise: 

Create a 'salary' field in our df. make it 30000 for mechanics, 40000 for salespeople and 70000 for managers.



In [87]:
def salary(word):
    if word == 'mechanic':
        return 30000
    elif word == 'sales':
        return 40000
    else:
        return 70000
salary('sales')    

40000

In [97]:
udf_salary = f.udf(salary)
udf_salary_typed = f.udf(salary, types.IntegerType())

In [98]:
df.withColumn('salary', udf_salary_typed(df['position'])).show()

+---+--------+------+
| id|position|salary|
+---+--------+------+
|  0| manager| 70000|
|  1|mechanic| 30000|
|  2|mechanic| 30000|
|  3| manager| 70000|
|  4|   sales| 40000|
|  5|mechanic| 30000|
|  6|mechanic| 30000|
|  7|mechanic| 30000|
|  8| manager| 70000|
|  9|mechanic| 30000|
| 10| manager| 70000|
| 11| manager| 70000|
+---+--------+------+



In [99]:
df.withColumn('salary', udf_salary_typed(df['position'])).printSchema()

root
 |-- id: long (nullable = true)
 |-- position: string (nullable = true)
 |-- salary: integer (nullable = true)



If we have a column that is not the desired type, we can convert it with `cast`.

In [96]:
df.withColumn('salary', udf_salary(df['position']).cast(types.IntegerType())).show()

+---+--------+------+
| id|position|salary|
+---+--------+------+
|  0| manager| 70000|
|  1|mechanic| 30000|
|  2|mechanic| 30000|
|  3| manager| 70000|
|  4|   sales| 40000|
|  5|mechanic| 30000|
|  6|mechanic| 30000|
|  7|mechanic| 30000|
|  8| manager| 70000|
|  9|mechanic| 30000|
| 10| manager| 70000|
| 11| manager| 70000|
+---+--------+------+



In [93]:
df.withColumn('salary', udf_salary(df['position']).cast(types.IntegerType())).printSchema()

root
 |-- id: long (nullable = true)
 |-- position: string (nullable = true)
 |-- salary: integer (nullable = true)



In [102]:
df = df.withColumn('salary', udf_salary_typed(df['position']))
df.show()

+---+--------+------+
| id|position|salary|
+---+--------+------+
|  0| manager| 70000|
|  1|mechanic| 30000|
|  2|mechanic| 30000|
|  3| manager| 70000|
|  4|   sales| 40000|
|  5|mechanic| 30000|
|  6|mechanic| 30000|
|  7|mechanic| 30000|
|  8| manager| 70000|
|  9|mechanic| 30000|
| 10| manager| 70000|
| 11| manager| 70000|
+---+--------+------+



### Summary statistics

https://databricks.com/blog/2015/06/02/statistical-and-mathematical-functions-with-dataframes-in-spark.html

In [103]:
df.stat.corr('id','salary')

0.20742413513940736

In [104]:
df.stat.cov('id','salary')

14999.999999999998

### .crosstab()

Crosstab returns the contingency table for two columns, as a DataFrame.

In [107]:
location_udf = f.udf(lambda: random.choice(['Madrid','Barcelona']))
df4 = df.withColumn('location',location_udf()).cache()

In [108]:
df4.show(5)

+---+--------+------+---------+
| id|position|salary| location|
+---+--------+------+---------+
|  0| manager| 70000|Barcelona|
|  1|mechanic| 30000|Barcelona|
|  2|mechanic| 30000|Barcelona|
|  3| manager| 70000|Barcelona|
|  4|   sales| 40000|   Madrid|
+---+--------+------+---------+
only showing top 5 rows



In [111]:
df4.crosstab('position','location').show()

+-----------------+---------+------+
|position_location|Barcelona|Madrid|
+-----------------+---------+------+
|            sales|        0|     1|
|          manager|        4|     1|
|         mechanic|        3|     3|
+-----------------+---------+------+



### Grouping

Grouping works very similarly to Pandas: executing groupby (or groupBy) on a DataFrame will return an object (a GroupedData) that can then be aggregated to obtain the results.

In [112]:
df4.groupBy('location').mean('salary').show()

+---------+------------------+
| location|       avg(salary)|
+---------+------------------+
|   Madrid|           40000.0|
|Barcelona|52857.142857142855|
+---------+------------------+



GroupedData has several aggregation functions defined:

In [114]:
stats = df4.groupBy('location').agg(f.mean('salary').alias('average'),
                                    f.stddev_pop('salary').alias('std'),
                                    f.count('salary').alias('count')
                                   
                                   ).show()

+---------+------------------+------------------+-----+
| location|           average|               std|count|
+---------+------------------+------------------+-----+
|   Madrid|           40000.0|15491.933384829668|    5|
|Barcelona|52857.142857142855| 19794.86637221574|    7|
+---------+------------------+------------------+-----+



We can do several aggregations in a single step, with a number of different syntaxes:

### Intersections

Ver much like SQL joins. We can specify the columns and the join method (left, right, inner, outer) or we can let Spark infer them.

In [116]:
random.seed(42)
data = list(zip([10,11,9,10,9],
                [5000,200,0,100],
                [random.choice(['Madrid','Barcelona','Sevilla']) for _ in range(5)]))

In [117]:
data

[(10, 5000, 'Sevilla'),
 (11, 200, 'Madrid'),
 (9, 0, 'Madrid'),
 (10, 100, 'Sevilla')]

In [118]:
new_df = session.createDataFrame(data,schema=['id','bonus','location'])
new_df.show()

+---+-----+--------+
| id|bonus|location|
+---+-----+--------+
| 10| 5000| Sevilla|
| 11|  200|  Madrid|
|  9|    0|  Madrid|
| 10|  100| Sevilla|
+---+-----+--------+



Spark refuses to do cross joins by default. To perform them, we can 

a) Allow then explicitly:

```python
session.conf.set("spark.sql.crossJoin.enabled", "true")
```

b) Specify the join criterion

```python
df4.join(new_df, on='id').show()
```

In [119]:
df4.join(new_df, on='id').show()

+---+--------+------+---------+-----+--------+
| id|position|salary| location|bonus|location|
+---+--------+------+---------+-----+--------+
| 10| manager| 70000|   Madrid| 5000| Sevilla|
| 11| manager| 70000|Barcelona|  200|  Madrid|
|  9|mechanic| 30000|   Madrid|    0|  Madrid|
| 10| manager| 70000|   Madrid|  100| Sevilla|
+---+--------+------+---------+-----+--------+



In [120]:
df4.join(new_df, on='id',how='left').show()

+---+--------+------+---------+-----+--------+
| id|position|salary| location|bonus|location|
+---+--------+------+---------+-----+--------+
|  0| manager| 70000|Barcelona| null|    null|
|  7|mechanic| 30000|   Madrid| null|    null|
|  6|mechanic| 30000|Barcelona| null|    null|
|  9|mechanic| 30000|   Madrid|    0|  Madrid|
|  5|mechanic| 30000|   Madrid| null|    null|
|  1|mechanic| 30000|Barcelona| null|    null|
| 10| manager| 70000|   Madrid| 5000| Sevilla|
| 10| manager| 70000|   Madrid|  100| Sevilla|
|  3| manager| 70000|Barcelona| null|    null|
|  8| manager| 70000|Barcelona| null|    null|
| 11| manager| 70000|Barcelona|  200|  Madrid|
|  2|mechanic| 30000|Barcelona| null|    null|
|  4|   sales| 40000|   Madrid| null|    null|
+---+--------+------+---------+-----+--------+



In [121]:
df4.join(new_df, on=df4['id']==new_df['id']).show()

+---+--------+------+---------+---+-----+--------+
| id|position|salary| location| id|bonus|location|
+---+--------+------+---------+---+-----+--------+
| 10| manager| 70000|   Madrid| 10| 5000| Sevilla|
| 11| manager| 70000|Barcelona| 11|  200|  Madrid|
|  9|mechanic| 30000|   Madrid|  9|    0|  Madrid|
| 10| manager| 70000|   Madrid| 10|  100| Sevilla|
+---+--------+------+---------+---+-----+--------+



#### Digression

We can monitor our running jobs and storage used at the Spark Web UI. We can get its url with sc.uiWebUrl.

StorageLevels represent how our DataFrame is cached: we can save the results of the computation up to that point, so that if we process several times the same data only the subsequent steps will be recomputed.

We can erase it with `unpersist`

#### Exercise

Calculate the [z-score](http://www.statisticshowto.com/probability-and-statistics/z-score/) of each employee's salary for their location

1) Calculate the mean and std of salary for each location

2) Annotate each employee with the stats corresponding to their location

3) Calculate the z-score

Note that we can build more complex boolean conditions for joining, as well as joining on columns that do not have the same name:

### Handling null values

## SQL querying

We need to register our DataFrame as a table in the SQL context in order to be able to query against it.

Once registered, we can perform queries as complex as we want.

#### Exercise:

replicate the previous exercise, but with SparkSQL instead of dataframe methods.

## Interoperation with Pandas

Easy peasy. We can convert a spark DataFrame into a Pandas one, which will `collect` it, and viceversa, which will distribute it.

## Writing out


#### Exercise

Repeat the exercise from the previous notebook, but this time with DataFrames.

Get stats for all tickets with destination MAD from `coupons150720.csv`.

You will need to extract ticket amounts with destination MAD, and then calculate:

1. Total ticket amounts per origin
2. Top 10 airlines by average amount

1) Extract the fields you need (c0,c1,c2,c3,c4 and c6) into a dataframe with proper names and types

Remember, you want to calculate:

Total ticket amounts per origin

Top 10 airlines by average amount

2) Total ticket amounts per origin

3) Top 10 Airlines by average amount



## Further Reading

https://databricks.com/blog/2016/07/14/a-tale-of-three-apache-spark-apis-rdds-dataframes-and-datasets.html

https://www.datacamp.com/community/tutorials/apache-spark-python

https://spark.apache.org/docs/2.2.0/sql-programming-guide.html

https://ogirardot.wordpress.com/2015/05/29/rdds-are-the-new-bytecode-of-apache-spark/

https://stackoverflow.com/questions/36822224/what-are-the-pros-and-cons-of-parquet-format-compared-to-other-formats

https://s3.amazonaws.com/assets.datacamp.com/blog_assets/PySpark_SQL_Cheat_Sheet_Python.pdf